In [4896]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer 

# Descripción del Notebook: Limpieza y Preparación del Dataset de Stack Overflow 2024

En este notebook se documenta el proceso completo de limpieza y preparación del dataset de la encuesta Stack Overflow 2023. Se llevaron a cabo las siguientes tareas principales:
- Imputación de Valores Faltantes: Se abordaron las celdas con valores NaN utilizando técnicas adecuadas para garantizar la integridad de los datos.
- Codificación de Variables Categóricas: Se transformaron las variables categóricas en un formato compatible con algoritmos de aprendizaje automático.
- Dataset Preparado para Modelado: El conjunto de datos final se dejó completamente listo para su integración en un modelo predictivo.

#### Este notebook sirvió como base para el desarrollo del modelo definitivo implementado en el pipeline del proyecto. No forma parte del código funcional del pipeline, sino que se incluye como una guía detallada del proceso paso a paso, documentando las decisiones tomadas y asegurando la reproducibilidad del trabajo.

Elimino primero el ResponseId (index), la columna de Check (pregunta en la encuesta para verificar que el encuestado presta atencion a lo que responde) y ConvertedCompYearly (conversion a dolares americanos de los que gana, lo elimino porque tratare solo EUR)

In [4897]:
# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

In [4898]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/data_2024.pickle', 'rb') as archivo:
    df = pickle.load(archivo)

In [4899]:
df = df.drop(columns=['Check', 'ConvertedCompYearly'])

### Empiezo a trabajar columna por columna con los ecoders. 
0. MainBranch: Que tipo de programador eres:
- I am a developer by profession
- I am not primarily a developer, but I write code sometimes as part of my work/studies 
- I used to be a developer by profession, but no longer am 
- I am learning to code 
- I code primarily as a hobby 
- None of these

Decido hacer un Label encoder, con valores del 0 (None of these) al 5 (Developer by profession), ya que quiero que se le de eventualmente mas peso a estos perfiles

In [4903]:
df['MainBranch'].unique()

array(['I am a developer by profession',
       'I am not primarily a developer, but I write code sometimes as part of my work/studies'],
      dtype=object)

In [4904]:
labels = {#'None of these': 0, 
        #   'I code primarily as a hobby': 1, 
        #   'I am learning to code':2,  --------------> todos estos son valores que aparecen en la encuesta, pero no en las respuestas
        #   'I used to be a developer by profession, but no longer am':3, 
          'I am not primarily a developer, but I write code sometimes as part of my work/studies': 4,
          'I am a developer by profession':5}
df['MainBranch'] = df['MainBranch'].map(labels).fillna(-1)

In [4905]:
df['MainBranch'].isna().sum()

0

1. Age:
- Under 18 years old 
- 18-24 years old 
- 25-34 years old 
- 35-44 years old 
- 45-54 years old 
- 55-64 years old 
- 65 years or older 
- Prefer not to say

Decido primero hacer un mapping y reducir las categorias, y luego hacer un OneHotEncoder: aunque se agregarian mas columnas, no son muchas.

Antes de hacer el OneHotEncoder verifico que no hayan nans. Si los hay, los trato primero, y leugo hago OneHot

In [4906]:
print(df['Age'].unique())

['25-34 years old' '35-44 years old' '45-54 years old' '18-24 years old'
 '55-64 years old' '65 years or older' 'Under 18 years old'
 'Prefer not to say']


In [4907]:
print(df['Age'].isna().sum())
print(df['Age'].dtype)
df['Age'] = df['Age'].fillna('Prefer not to say')  # Rellenar NaN con una categoría válida
df['Age'] = df['Age'].str.strip()   # por las dudas, elimino espacios vacios que me puedan causar problemas con el One Hot

0
object


In [4908]:
df['Age'].value_counts()

Age
25-34 years old       318
35-44 years old       258
45-54 years old        99
18-24 years old        87
55-64 years old        19
65 years or older       1
Under 18 years old      1
Prefer not to say       1
Name: count, dtype: int64

In [4909]:
age_mapping = {
    'Under 18 years old': 'Joven',
    '18-24 years old': 'Joven',
    '25-34 years old': 'Adulto',
    '35-44 years old': 'Adulto',
    '45-54 years old': 'Adulto',
    '55-64 years old': 'Senior',
    '65 years or older': 'Senior',
    'Prefer not to say': 'Desconocido'
}

df['Age_Grouped'] = df['Age'].map(age_mapping)

In [4910]:
# Aplicar one-hot encoding a 'Age_Grouped'
most_frequent_age = df['Age'].mode()[0]
df['Age'].fillna(most_frequent_age, inplace=True)

encoder = OneHotEncoder(sparse_output=False)
encoded_age = encoder.fit_transform(df[['Age_Grouped']])
encoded_age_df = pd.DataFrame(encoded_age, columns=encoder.get_feature_names_out(['Age_Grouped']), index=df.index)
df = pd.concat([df, encoded_age_df], axis=1)

# Eliminar la columna original 'Age' y la columna 'Age_Grouped'
df.drop(['Age', 'Age_Grouped'], axis=1, inplace=True)

C:\Users\emita\AppData\Local\Temp\ipykernel_2432\734129936.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(most_frequent_age, inplace=True)


Compruebo que todo esta correcto, se han agregado las columnas correspondientes, y las filas siguen igual. Sigo. 

2. Employment:
- Employed, full-time 
- Employed, part-time 
- Independent contractor, freelancer, or self-employed
- Not employed, but looking for work 
- Not employed, and not looking for work 
- Student, full-time 
- Student, part-time 
- Retired 
- I prefer not to say

Misma justificación que para el sof_23

In [4911]:
df['Employment'].unique()

array(['Employed, full-time',
       'Employed, full-time;Independent contractor, freelancer, or self-employed',
       'Independent contractor, freelancer, or self-employed',
       'Employed, full-time;Student, part-time',
       'Student, part-time;Employed, part-time',
       'Student, full-time;Employed, part-time',
       'Employed, full-time;Independent contractor, freelancer, or self-employed;Student, part-time',
       'Not employed, but looking for work;Independent contractor, freelancer, or self-employed',
       'Employed, full-time;Not employed, but looking for work',
       'Employed, part-time',
       'Employed, full-time;Student, full-time;Independent contractor, freelancer, or self-employed',
       'Independent contractor, freelancer, or self-employed;Employed, part-time',
       'Employed, full-time;Independent contractor, freelancer, or self-employed;Employed, part-time',
       'Employed, full-time;Student, full-time',
       'Employed, full-time;Employed, part-ti

In [4912]:
df['Employment'].value_counts()

Employment
Employed, full-time                                                                             609
Independent contractor, freelancer, or self-employed                                             65
Employed, full-time;Independent contractor, freelancer, or self-employed                         48
Employed, full-time;Student, part-time                                                           23
Employed, full-time;Student, full-time                                                            7
Student, part-time;Employed, part-time                                                            6
Employed, part-time                                                                               6
Independent contractor, freelancer, or self-employed;Employed, part-time                          6
Employed, full-time;Independent contractor, freelancer, or self-employed;Student, part-time       3
Student, full-time;Employed, part-time                                                   

In [4913]:
# Reemplazo 'Retired' para no crear una nueva columna con eso, al ser 1 solo valor, no tendra importancia. 
df['Employment'] = df['Employment'].replace('Retired', 'I prefer not to say')

In [4914]:
# Crear las columnas binarias
df['is_full_time'] = df['Employment'].str.contains('Employed, full-time').fillna(False).astype(int)
df['is_part_time'] = df['Employment'].str.contains('Employed, part-time').fillna(False).astype(int)
df['is_independent'] = df['Employment'].str.contains('Independent contractor, freelancer, or self-employed').fillna(False).astype(int)

# Crear la columna 'num_jobs'
df['num_jobs'] = df['Employment'].str.split(';').str.len().fillna(0).astype(int)

# Tratar los valores especiales (Retired, I prefer not to say, etc.)
df['is_other_employment'] = ((df['is_full_time'] == 0) & (df['is_part_time'] == 0) &
                            (df['is_independent'] == 0)).astype(int)
# Eliminar la columna original 'Employment'
df.drop('Employment', axis=1, inplace=True)

# Visualizar el DataFrame resultante
df.head()

,MainBranch,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,TechDoc,YearsCode,YearsCodePro,DevType,OrgSize,PurchaseInfluence,BuyNewTool,BuildvsBuy,TechEndorse,Country,Currency,CompTotal,LanguageHaveWorkedWith,LanguageWantToWorkWith,LanguageAdmired,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,DatabaseAdmired,PlatformHaveWorkedWith,PlatformWantToWorkWith,PlatformAdmired,WebframeHaveWorkedWith,WebframeWantToWorkWith,WebframeAdmired,EmbeddedHaveWorkedWith,EmbeddedWantToWorkWith,EmbeddedAdmired,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,MiscTechAdmired,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,ToolsTechAdmired,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,NEWCollabToolsAdmired,OpSysPersonal use,OpSysProfessional use,OfficeStackAsyncHaveWorkedWith,OfficeStackAsyncWantToWorkWith,OfficeStackAsyncAdmired,OfficeStackSyncHaveWorkedWith,OfficeStackSyncWantToWorkWith,OfficeStackSyncAdmired,AISearchDevHaveWorkedWith,AISearchDevWantToWorkWith,AISearchDevAdmired,NEWSOSites,SOVisitFreq,SOAccount,SOPartFreq,SOHow,SOComm,AISelect,AISent,AIBen,AIAcc,AIComplex,AIToolCurrently Using,AIToolInterested in Using,AIToolNot interested in Using,AINextMuch more integrated,AINextNo change,AINextMore integrated,AINextLess integrated,AINextMuch less integrated,AIThreat,AIEthics,AIChallenges,TBranch,ICorPM,WorkExp,Knowledge_1,Knowledge_2,Knowledge_3,Knowledge_4,Knowledge_5,Knowledge_6,Knowledge_7,Knowledge_8,Knowledge_9,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Frustration,ProfessionalTech,ProfessionalCloud,ProfessionalQuestion,Industry,JobSatPoints_1,JobSatPoints_4,JobSatPoints_5,JobSatPoints_6,JobSatPoints_7,JobSatPoints_8,JobSatPoints_9,JobSatPoints_10,JobSatPoints_11,SurveyLength,SurveyEase,JobSat,Age_Grouped_Adulto,Age_Grouped_Desconocido,Age_Grouped_Joven,Age_Grouped_Senior,is_full_time,is_part_time,is_independent,num_jobs,is_other_employment
0,5,Remote,I don’t code outside of work,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",On the job training;Other online resources (e....,Technical documentation;Books;Written Tutorial...,API document(s) and/or SDK document(s);Traditi...,14,10,DevOps specialist,500 to 999 employees,I have some influence,Start a free trial;Ask developers I know/work ...,Is ready-to-go but also customizable for growt...,Quality support system;Reputation for quality ...,Spain,EUR European Euro,80000.0,Bash/Shell (all shells);Python;SQL,Go,NaN,NaN,NaN,NaN,OpenStack,NaN,NaN,Django;Flask,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,APT;Docker;Kubernetes;Pip;Terraform,Kubernetes,Kubernetes,Vim;Visual Studio Code,NaN,NaN,Android;Ubuntu,Ubuntu,GitHub Discussions;Jira,NaN,NaN,Google Meet;Matrix;Mattermost,NaN,NaN,NaN,NaN,NaN,Stack Overflow;Stack Exchange,A few times per month or weekly,Yes,Less than once per month or monthly,Quickly finding code solutions,"Yes, somewhat","No, but I plan to soon",Favorable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Circulating misinformation or disinformation;M...,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,NaN,1.0,0.0,0.0,0.0,1,0,0,1,0
1,5,Remote,I don’t code outside of work,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;On the job training;Oth...,Technical documentation;Blogs;Books;Stack Over...,API document(s) and/or SDK document(s);User gu...,15,11,Engineering manager,10 to 19 employees,I have a great deal of influence,Start a free trial;Ask developers I know/work ...,Is ready-to-go but also customizable for growt...,APIs;Customization;Quality support system;Repu...,Spain,EUR European Euro,85000.0,Bash/Shell (all shells);HTML/CSS;JavaScript;Py...,Bash/Shell (all shells);Go;HTML/CSS;JavaScript...,Bash/Shell (all shells);HTML/CSS;JavaScript;Py...,MariaDB;MongoDB;MySQL;SQLite,Cockroachdb;MariaDB;MySQL;PostgreSQL;SQLite,MariaDB;MySQL;SQLite,Amazon Web Services (AWS);Cloudflare;Google Cl...,Amazon Web Services (AWS);Cloudflare;

3. RemoteWork:
- Remote 
- In-person 
- Hybrid (some remote, some in-person)

Misma justificación que sof_23

In [4915]:
df['RemoteWork'].isna().sum()

0

In [4916]:
df['RemoteWork'].value_counts()

RemoteWork
Remote                                  421
Hybrid (some remote, some in-person)    276
In-person                                87
Name: count, dtype: int64

In [4917]:
df['Remote_grouped'] = df['RemoteWork'].map({'Remote': 'Remote/Hybrid', 'Hybrid (some remote, some in-person)': 'Remote/Hybrid', 'In-person': 'In-person'})

# Aplicar one-hot encoding a 'Remote_grouped'
encoder = OneHotEncoder(sparse_output=False)
encoded_remote = encoder.fit_transform(df[['Remote_grouped']])
encoded_remote_df = pd.DataFrame(encoded_remote, columns=encoder.get_feature_names_out(['Remote_grouped']), index=df.index)
df = pd.concat([df, encoded_remote_df], axis=1)

# Eliminar las columnas originales
df.drop(['RemoteWork', 'Remote_grouped'], axis=1, inplace=True)

4. CodingActivities (fuera del trabajo):
- Hobby (2)
- Freelance/contract work (3)
- Contribute to open-source projects (4) 
- Bootstrapping a business (5)
- School or academic work (1)
- Professional development or self-paced learning from online courses (6)
- I don’t code outside of work (0)
- Other (-1)

Decido un Label encoder y darle mas peso a unas categorias, mas que a otras (indicado el peso de cada una)

Hay Muchos Nan, los trabajo imputandolos a la categoria de "Other"

In [4918]:
df['CodingActivities'].isna().sum()

0

In [4923]:
def process_multiple_categories(df, category_column, target_column, separator=','):
    """
    Realiza target encoding para columnas con múltiples valores separados por un delimitador.
    """
    # Llenar valores NaN en la columna categórica con 'Unknown'
    df[category_column] = df[category_column].fillna('Unknown')
    
    # Expandir los valores separados por comas en listas
    df[category_column] = df[category_column].apply(lambda x: x.split(separator) if isinstance(x, str) else [x])
    
    # Crear un DataFrame temporal para aplanar las listas
    exploded_df = df.explode(category_column)
    
    # Calcular el promedio del target por categoría
    target_map = exploded_df.groupby(category_column)[target_column].mean().to_dict()
    
    # Función para calcular el promedio de los valores codificados para una fila
    def calculate_row_encoding(categories):
        encoded_values = [target_map.get(cat, 0) for cat in categories]  # 0 para categorías desconocidas
        return sum(encoded_values) / len(encoded_values) if encoded_values else 0
    
    # Crear la nueva columna con el promedio del encoding
    df[f'{category_column}_encoded'] = df[category_column].apply(calculate_row_encoding)
    
    return df, target_map


# Aplicar el procesamiento a la columna 'LearnCodeOnline_Category'
df, target_map = process_multiple_categories(df, 'CodingActivities', 'CompTotal')

# Eliminamos las columnas temporales si ya no son necesarias
df = df.drop(['CodingActivities'], axis=1, errors='ignore')

# Revisamos los valores codificados
print("\nValores únicos después del encoding:")
print(df[f'CodingActivities_encoded'].value_counts(dropna=False))


Valores únicos después del encoding:
CodingActivities_encoded
52676.915584     154
57583.196429     112
50661.553398     103
62039.615385      52
47831.547619      42
61059.714286      35
68757.166667      24
57660.869565      23
54517.647059      17
21988.235294      17
40400.000000      17
39988.875000      16
84163.076923      13
97400.000000      10
49000.000000      10
127777.777778      9
59222.222222       9
43500.000000       8
80828.571429       7
43542.857143       7
32725.714286       7
50285.714286       7
78500.000000       7
54175.000000       5
86600.000000       5
70760.000000       5
88200.000000       5
52550.000000       4
69750.000000       4
69250.000000       4
67750.000000       4
44375.000000       4
84666.666667       3
38333.333333       3
102666.666667      3
82666.666667       3
51000.000000       3
75000.000000       3
59333.333333       3
57500.000000       2
32500.000000       2
35000.000000       2
82000.000000       1
41000.000000       1
40000.000000 

5. EdLevel: 
- Primary/elementary school (0)
- Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.) (1)
- Some college/university study without earning a degree (2)
- Associate degree (A.A., A.S., etc.) (3)
- Bachelor’s degree (B.A., B.S., B.Eng., etc.) (4) 
- Master’s degree (M.A., M.S., M.Eng., MBA, etc.) (5) 
- Professional degree (JD, MD, Ph.D, Ed.D, etc.) (6)
- Something else (-1)

Hago un LabelEncoder con los valores destacados

In [4924]:
df['EdLevel'].unique()

array(['Master’s degree (M.A., M.S., M.Eng., MBA, etc.)',
       'Bachelor’s degree (B.A., B.S., B.Eng., etc.)',
       'Professional degree (JD, MD, Ph.D, Ed.D, etc.)',
       'Some college/university study without earning a degree',
       'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
       'Associate degree (A.A., A.S., etc.)', 'Something else',
       'Primary/elementary school'], dtype=object)

In [4925]:
df['EdLevel'].isna().sum()

0

In [4926]:
labels5 = {
    'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)': 5,
    'Bachelor’s degree (B.A., B.S., B.Eng., etc.)': 4,
    'Professional degree (JD, MD, Ph.D, Ed.D, etc.)': 6,
    'Some college/university study without earning a degree': 2, 
    'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 1,
    'Associate degree (A.A., A.S., etc.)': 3, 
    'Something else': -1,
    'Primary/elementary school': 0
}
df['EdLevel'] = df['EdLevel'].map(labels5).fillna(-1)

6. LearnCode: 
- Books / Physical media (3): Implica un esfuerzo autodirigido, pero es formal y metódico.
- Coding Bootcamp (3): También estructurado, pero más intensivo y con un enfoque en habilidades prácticas.
- Colleague (2): Más relacionado con la práctica laboral, pero también limitado al entorno de trabajo.
- Friend or family member (1): Aprendizaje menos formal y probablemente limitado en alcance.
- Online Courses or Certification (2): Similar al entrenamiento en el trabajo, pero autodirigido y menos formal.
- On the job training (2): Aprender en el trabajo es práctico, pero menos estructurado.
- Other online resources (e.g., videos, blogs, forum, online community) (2): Aunque útil, suele ser más informal y variado en calidad.
- School (i.e., University, College, etc) (4): Representa una forma tradicional de aprendizaje, estructurada y generalmente reconocida como valiosa.
- Other (please specify):(0)

Le diversa importancia a las categorias con total discrecion (segun el tipo de educacion). Un OneHot me crearia demasiadas columnas. 

Prefiero hacer una funcion que, luego de dividir en el ";", se reemplace cada categoria por el numero correspondiente, y luego sumo esos numeros. De esta forma, una persona que se ha educado de 5 fuentes diversas, tendra mas puntaje que una que solo se ha educado con 1. Da cuenta del compromiso y profesionalismo que tiene esa persona. 

In [4933]:
# Aplicar el procesamiento a la columna 'LearnCodeOnline_Category'
df, target_map = process_multiple_categories(df, 'LearnCode', 'CompTotal')

# Eliminamos las columnas temporales si ya no son necesarias
df = df.drop(['LearnCode'], axis=1, errors='ignore')

# Revisamos los valores codificados
print("\nValores únicos después del encoding:")
print(df[f'LearnCode_encoded'].value_counts(dropna=False))


Valores únicos después del encoding:
LearnCode_encoded
56522.524866    45
57016.707046    34
54642.999586    33
56139.794611    30
55117.292307    29
                ..
54444.679897     1
52661.729082     1
57877.632473     1
53440.767088     1
21000.000000     1
Name: count, Length: 145, dtype: int64


7. LearnCodeOnline
- Stack Overflow ('Community_Collaboration')
- Blogs ('Community_Collaboration')
- Online community (including social media) ('Community_Collaboration')
- Video-based online courses ('Visual_Interactive')
- Online challenges (e.g., daily or weekly coding challenges) ('Community_Collaboration')
- Coding sessions (live or recorded) ('Community_Collaboration')
- Written-based online courses ('Technical_Reference')
- How-to videos ('Visual_Interactive')
- Auditory material (e.g., podcasts) ('Visual_Interactive')
- Technical documentation ('Technical_Reference')
- Certification videos ('Visual_Interactive')
- Books ('Technical_Reference')
- Games that teach programming ('Visual_Interactive')
- AI chatbot or other tool ('Community_Collaboration')
- Written tutorials ('Technical_Reference')
- Interactive tutorial ('Visual_Interactive')
- Other ('Visual_Interactive')

Estas herramientas no tienen mas importancia unas que otras, ni son ordinales. Elijo usar OneHotEncoder. Como son muchas columnas, decido agruparlas, para que no sean tantas.
Dejo 3 categorias: 'Visual_Interactive', 'Technical_Reference', 'Community_Collaboration'. 

In [4934]:
df['LearnCodeOnline'].unique()

array(['Technical documentation;Books;Written Tutorials;Stack Overflow;Written-based Online Courses',
       'Technical documentation;Blogs;Books;Stack Overflow;Coding sessions (live or recorded);Interactive tutorial;Video-based Online Courses',
       'Technical documentation;Blogs;Books;Written Tutorials;Stack Overflow;Social Media;How-to videos;Interactive tutorial;Video-based Online Courses;Written-based Online Courses;Online challenges (e.g., daily or weekly coding challenges);Programming Games',
       'Technical documentation;Books;Video-based Online Courses', nan,
       'Technical documentation;Written Tutorials;How-to videos',
       'Technical documentation;Stack Overflow;How-to videos;AI',
       'Technical documentation;Books;Written Tutorials;Stack Overflow;Coding sessions (live or recorded);AI;Online challenges (e.g., daily or weekly coding challenges)',
       'Technical documentation;Blogs;Books;Written Tutorials;Stack Overflow;Coding sessions (live or recorded);Social

Creo una funcion para ver los valores unicos, ya que en las celdas hay muchos valores. 

In [4939]:
# Aplicar el procesamiento a la columna 'LearnCodeOnline_Category'
df, target_map = process_multiple_categories(df, 'LearnCodeOnline', 'CompTotal')

# Eliminamos las columnas temporales si ya no son necesarias
df = df.drop(['LearnCodeOnline'], axis=1, errors='ignore')

# Revisamos los valores codificados
print("\nValores únicos después del encoding:")
print(df[f'LearnCodeOnline_encoded'].value_counts(dropna=False))


Valores únicos después del encoding:
LearnCodeOnline_encoded
59914.532164    171
50000.000000     18
55000.000000     17
60000.000000     15
40000.000000     14
               ... 
60585.652174      1
65142.500000      1
66172.000000      1
36642.500000      1
37636.433673      1
Name: count, Length: 255, dtype: int64


8. TechDoc
- API document(s) and/or SDK document(s) 
- User guides or README files found in the source repository 
- First-party knowledge base 
- Traditional public search engine
- AI-powered search/dev tool (free) 
- AI-powered search/dev tool (paid)
- Other 

Separar por comas y OneHotEncoder  (lo hago en el codigo del final, con el resto de las columnas donde hago esto.)
Los Nan los dejo asi, para que simplemente tenga 0 en todas las columnas creadas con el OneHot. 

In [4940]:
df['TechDoc'].isnull().sum()

262

9. YearsCode: sacar los nan y pasarlo a int

In [4942]:
df['YearsCode'].isnull().sum()
moda = df['YearsCode'].mode()[0]
df['YearsCode'] = df['YearsCode'].fillna(moda)

In [4943]:
df['YearsCode']=df['YearsCode'].astype(int)

In [4944]:
df['YearsCode'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 784 entries, 0 to 783
Series name: YearsCode
Non-Null Count  Dtype
--------------  -----
784 non-null    int32
dtypes: int32(1)
memory usage: 3.2 KB


10. YearsCodePro: reemplazo nan y paso a int

In [4945]:
df['YearsCodePro'].isnull().sum()

5

In [4946]:
moda = df['YearsCodePro'].mode()[0]
df['YearsCodePro'] = df['YearsCodePro'].fillna(moda)

In [4947]:
df['YearsCodePro'].unique()

array(['10', '11', '24', '22', '31', '14', '2', '7', '21', '28', '6', '4',
       '16', '8', '9', '1', '26', '12', '20', '23', '13', '3', '25', '5',
       '18', '33', '32', 'Less than 1 year', '15', '27', '17', '30', '19',
       '29', '38', '44', '39', '37', '45', '34', '35'], dtype=object)

In [4948]:
df['YearsCodePro']=df['YearsCodePro'].replace('Less than 1 year', 0)

In [4949]:
df['YearsCodePro']=df['YearsCodePro'].astype(int)
df['YearsCodePro'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 784 entries, 0 to 783
Series name: YearsCodePro
Non-Null Count  Dtype
--------------  -----
784 non-null    int32
dtypes: int32(1)
memory usage: 3.2 KB


11. DevType
- Academic researcher (5)
- Blockchain (6)
- Cloud infrastructure engineer (3)
- Data or business analyst (2)
- Data engineer (2)
- Data scientist or machine learning specialist (2)
- Database administrator (7)
- Designer (7)
- Developer Advocate (1)
- Developer, AI (1)
- Developer, back-end (1)
- Developer, desktop or enterprise applications (1)
- Developer, embedded applications or devices (1)
- Developer Experience (1)
- Developer, front-end (1)
- Developer, full-stack(1)
- Developer, game or graphics (1)
- Developer, mobile(1) 
- Developer, QA or test (1)
- DevOps specialist (3)
- Educator (5)
- Engineer, site reliability (3)
- Engineering manager (4)
- Hardware Engineer (6)
- Marketing or sales professional (7)
- Product manager (4)
- Project manager (4)
- Research & Development role (5)
- Scientist (5)
- Senior Executive (C-Suite, VP, etc.) (4)
- Student (7)
- System administrator (3)
- Security professional (3)
- Other (7)

Elijo agruparlos para que no sean tantos, y asi simplificar el analisis, y luego un OneHotEncoder:
1. Gestión y Liderazgo
2. Desarrollo de Software
3. Infraestructura y Sistemas
4. Ingeniería de Datos y Ciencia de Datos
5. Investigación y Educación
6. Hardware y Blockchain
7. Roles de Soporte y Otros

In [4950]:
# Primero me fijo que no hayan Nan
df['DevType'].isnull().sum()

4

In [4955]:
# Aplicar el procesamiento a la columna 'DevType_encoded'
df, target_map = process_multiple_categories(df, 'DevType', 'CompTotal')

# Eliminamos las columnas temporales si ya no son necesarias
df = df.drop(['DevType'], axis=1, errors='ignore')

# Revisamos los valores codificados
print("\nValores únicos después del encoding:")
print(df[f'DevType_encoded'].value_counts(dropna=False))


Valores únicos después del encoding:
DevType_encoded
51504.565060     227
55423.749711     184
53397.220745      47
64741.897444      41
49954.138298      32
76391.304348      23
36637.095238      21
58404.761905      21
51176.190476      21
41373.611111      18
62797.777778      18
70387.500000      16
46601.906155      14
90615.384615      13
70170.909091      11
50224.763298       8
150428.571429      7
50166.666667       6
47166.666667       6
31383.333333       6
49553.096631       6
55444.763298       5
92200.000000       5
84200.000000       5
52250.000000       4
55750.000000       4
101250.000000      4
60952.500000       4
130000.000000      3
43000.000000       2
12000.000000       1
36000.000000       1
Name: count, dtype: int64


12. OrgSize: Cuántas personas forman parte de la organización para la que trabajas (organización primaria)
- I don’t know
- Just me - I am a freelancer, sole proprietor, etc. 
- 2 to 9 employees 
- 10 to 19 employees 
- 20 to 99 employees 
- 100 to 499 employees 
- 500 to 999 employees 
- 1000 to 4999 employees 
- 5000 to 9999 employees 
- 10000 or more employees 

Label Encoder, en ese orden

In [4956]:
df['OrgSize'].unique()

array(['500 to 999 employees', '10 to 19 employees', '20 to 99 employees',
       '100 to 499 employees', '5,000 to 9,999 employees',
       '1,000 to 4,999 employees', '2 to 9 employees',
       '10,000 or more employees',
       'Just me - I am a freelancer, sole proprietor, etc.',
       'I don’t know', nan], dtype=object)

In [4960]:
# Agrupación de categorías
orgsize_mapping = {
    'I don’t know': 'Desconocido',
    'Just me - I am a freelancer, sole proprietor, etc.': 'Muy Pequeña',
    '2 to 9 employees': 'Muy Pequeña',
    '10 to 19 employees': 'Muy Pequeña',
    '20 to 99 employees': 'Pequeña',
    '100 to 499 employees': 'Pequeña',
    '500 to 999 employees': 'Mediana',
    '1,000 to 4,999 employees': 'Mediana',
    '5,000 to 9,999 employees': 'Grande',
    '10,000 or more employees': 'Grande'
}
df['OrgSize_Grouped'] = df['OrgSize'].map(orgsize_mapping).fillna('Desconocido')

# One-hot encoding de la variable agrupada
encoder = OneHotEncoder(sparse_output=False)
encoded_orgsize = encoder.fit_transform(df[['OrgSize_Grouped']])
encoded_orgsize_df = pd.DataFrame(encoded_orgsize, columns=encoder.get_feature_names_out(['OrgSize_Grouped']), index=df.index)
df = pd.concat([df, encoded_orgsize_df], axis=1)

# Eliminar las columnas originales
df.drop(['OrgSize', 'OrgSize_Grouped'], axis=1, inplace=True)

13. PurchaseInfluence: Nivel de influencia sobre nuevas tecnologías adquiridas por la organización
- I have little or no influence 
- I have some influence 
- I have a great deal of influence

Label Encoder en ese orden

In [4961]:
# Primero trato los Nan
df['PurchaseInfluence'] = df['PurchaseInfluence'].fillna('Other')
print(df['PurchaseInfluence'].isna().sum())

0


In [4962]:
labels13 = {
    'I have little or no influence': 0,
    'I have some influence': 1,
    'I have a great deal of influence': 2,
    'Other': -1
}
df['PurchaseInfluence'] = df['PurchaseInfluence'].map(labels13).fillna(-1)

14. BuyNewTool: Cómo descubrir o investigas soluciones disponibles en nuevas herramientas o softwares
- Start a free trial 
- Ask developers I know/work with 
- Ask a generative AI tool 
- Visit developer communities like Stack Overflow 
- Read ratings or reviews on third party sites like G2 Crowd 
- Research companies that have advertised on sites I visit 
- Research companies that have emailed me
- Other 

No tienen orden, separo por ; y hago OneHotEncoder al final

In [4963]:
df['BuyNewTool'].isna().sum()
df['BuyNewTool'] = df['BuyNewTool'].fillna('Other')

15. BuildvsBuy: Irrelevante, la elimino. Además, no está en el df 2023. 

In [4964]:
df = df.drop(['BuildvsBuy'], axis=1)

16. TechEndorse: Que caracteristica crees más importante a la hora de adquirir una nueva tecnología
- APIs 
- Customization
- Integrated developer community 
- Connected to an open source project 
- Quality support system AI tool integration 
- Reputation for quality and excellence 
- Frequent updates to features
- Other

Divido por ; u hago OneHotEncoder. Todo al final 

In [4965]:
df['TechEndorse']

0      Quality support system;Reputation for quality ...
1      APIs;Customization;Quality support system;Repu...
2                                                    NaN
3                  Reputation for quality and excellence
4      Quality support system;Connected to an open so...
                             ...                        
779                       Integrated developer community
780                                                  NaN
781    APIs;Customization;Integrated developer commun...
782    APIs;Customization;Integrated developer commun...
783                                                  NaN
Name: TechEndorse, Length: 784, dtype: object

In [4966]:
# Primero trato los Nan
df['TechEndorse'].isna().sum()
df['TechEndorse'] = df['TechEndorse'].fillna('Other')

17. Country:
Si hago solo de Spain, la elimino. 

18. Currency: Si hago solo de Europa o Spain, la elimino 

In [4968]:
df = df.drop(['Country', 'Currency'], axis=1)

#### Ahora vienen una seguidilla de columnas que tengo que separar por el ';' y luego hacer un OneHotEncoder. 

Creo una funcion para aplicar lo mismo a todas juntas. El codigo de estas columnas se vera al final.

Luego elimino las que indico en el listado, por no aportar informacion nueva ni relevante

19. LanguageHaveWorkedWith: Dividirlo por ';' y luego OneHotEncoder. Se haran muchas columnas, pero no queda otra. 

20. LanguageWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

21. LanguageAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

22. DatabaseHaveWorkedWith: Igual

23. DatabaseWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

24. DatabaseAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

25. PlatformHaveWorkedWith: Igual

26. PlatformWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

27. PlatformAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

28. WebframeHaveWorkedWith: Igual

29. WebframeWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

30. WebframeAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

31. EmbeddedHaveWorkedWith: Igual 

32. EmbeddedWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas 

33. EmbeddedAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas 

34. MiscTechHaveWorkedWith: Igual 

35. MiscTechWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas 

36. MiscTechAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas 

37. ToolsTechHaveWorkedWith: Igual 

38. ToolsTechWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

39. ToolsTechAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

40. NEWCollabToolsHaveWorkedWith: Igual 

41. NEWCollabToolsWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

42. NEWCollabToolsAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

43. OpSysPersonal: La elimino, me quedo solo con el uso de sistema operativo profesional. 

44. OpSysProfessional use: Agrupo los sistemas operativos en los mas representativos para no hacer tantas columnas: 
- AIX (4) Otros
- Android (3) Android-based
- Arch (1)
- BSD (1)
- ChromeOS (3)
- Cygwin (4)
- Debian (1) Linux-based
- Fedora (1)
- Haiku (1)
- iOS (2) MacOs
- iPadOS (2)
- MacOS (2)
- Other Linux-based (1)
- Red Hat (1)
- Solaris (1)
- Ubuntu (1)
- Windows (0) Windows-based
- Windows Subsystem for Linux (WSL) (0)
- Other (4)

Una vez agrupadas, le hago un MultilabelEncoder para poder poner varias etiquetas por fila

In [4969]:
# Primero trato los Nan
df['OpSysProfessional use'] = df['OpSysProfessional use'].fillna('Other')
df['OpSysProfessional use'].isnull().sum()

0

In [4970]:
# Diccionario de mapeo
os_map = {
    'AIX': 'Otros',
    'Android': 'Android-based',
    'Arch': 'Linux-based',
    'BSD': 'Linux-based',
    'ChromeOS': 'Android-based',
    'Cygwin': 'Otros',
    'Debian': 'Linux-based',
    'Fedora': 'Linux-based',
    'Haiku': 'Linux-based',
    'iOS': 'MacOS',
    'iPadOS': 'MacOS',
    'MacOS': 'MacOS',
    'Other Linux-based': 'Linux-based',
    'Red Hat': 'Linux-based',
    'Solaris': 'Linux-based',
    'Ubuntu': 'Linux-based',
    'Windows': 'Windows-based',
    'Windows Subsystem for Linux (WSL)': 'Linux-based',
    'Other': 'Otros'
}

In [4971]:
# Función modificada para manejar NaN
def split_and_map(value):
    if pd.isna(value):
        return []  # Retorna una lista vacía para NaN
    return [os_map.get(os.strip(), 'Otros') for os in value.split(';')]

# Aplicar la función a la columna
df['OpSysProfessional_Category'] = df['OpSysProfessional use'].apply(split_and_map)

# Usar MultiLabelBinarizer para el one-hot encoding
mlb = MultiLabelBinarizer()
encoded = mlb.fit_transform(df['OpSysProfessional_Category'])

# Crear DataFrame con las columnas codificadas
encoded_df = pd.DataFrame(encoded, columns=mlb.classes_, index=df.index)

# Unir el DataFrame original con las nuevas columnas codificadas
df = pd.concat([df, encoded_df], axis=1)

# Eliminar las columnas originales si ya no las necesitas
df = df.drop(['OpSysProfessional_Category', 'OpSysProfessional use'], axis=1)

# Verificar que no se haya creado una columna 'nan'
if 'nan' in encoded_df.columns:
    df = df.drop('nan', axis=1)

45. OfficeStackAsyncHaveWorkedWith (Herramientas de documentación que has usado): Igual 

46. OfficeStackAsyncWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

47. OfficeStackAsyncAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

48. OfficeStackSyncHaveWorkedWith(Herramientas de comunicación que has usado): Igual

49. OfficeStackSyncWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

50. OfficeStackSyncAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas 

51. AISearchDevHaveWorkedWith: Igual

52. AISearchDevWantToWorkWith: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

53. AISearchDevAdmired: La elimino, dejo solo la de "HaveWorkedWith" como representativa. Sino se me hacen muchas columnas y muy repetitivas

54. NEWSOSites(Uso de StackOverFlow): No tiene relevancia para mi analisis en concreto, lo elimino. 

55. SOVisitFreq: No tiene relevancia para mi analisis en concreto, lo elimino.

56. SOAccount: No tiene relevancia para mi analisis en concreto, lo elimino.

57. SOPartFreq: No tiene relevancia para mi analisis en concreto, lo elimino.

58. SOHow: No tiene relevancia para mi analisis en concreto, lo elimino.

59. SOComm: No tiene relevancia para mi analisis en concreto, lo elimino.

60. AISelect (Usas IA para tu proceso de desarrollo):
- Yes 
- No, but I plan to soon 
- No, and I don't plan to
Elijo Label encoder con (2) para el Yes.

In [4972]:
df['AISelect'].unique()

array(['No, but I plan to soon', 'Yes', "No, and I don't plan to", nan],
      dtype=object)

In [4973]:
# Primero trato los Nan
df['AISelect'] = df['AISelect'].fillna('Other')
print(df['AISelect'].isna().sum())

0


In [4974]:
labels60 = {
    "No, and I don't plan to": 0,
    'No, but I plan to soon': 1,
    'Yes': 2,
    'Other': -1
}
df['AISelect'] = df['AISelect'].map(labels60).fillna(-1)

61. AISent (Uso de herramientas de IA como parte de su flujo de trabajo de desarrollo): 
- Very favorable (5) 
- Favorable(4)
- Indifferent (3)
- Unfavorable (2)
- Very unfavorable (1)
- Unsure (0)

LabelEncoder con los valores indicados. 

In [4975]:
df['AISent'].unique()

array(['Favorable', 'Unsure', 'Indifferent', 'Very favorable', nan,
       'Unfavorable', 'Very unfavorable'], dtype=object)

In [4976]:
# Primero trato los Nan
df['AISent'] = df['AISent'].fillna('Unsure')
print(df['AISent'].isna().sum())

0


In [4977]:
labels61 = {
    'Very favorable': 5, 
    'Favorable': 4, 
    'Indifferent': 3, 
    'Unfavorable': 2,
    'Very unfavorable': 1,
    'Unsure': 0
}
df['AISent'] = df['AISent'].map(labels61).fillna(-1)

62. AIBen (beneficios importantes espera lograr con la IA): 
- Increase productivity 
- Greater efficiency 
- Make workload more manageable 
- Improve collaboration 
- Speed up learning 
- Improve accuracy in coding 
- None of the above 
- Other 

Dividirlo por ';' y luego OneHotEncoder. 

63. AIAcc (Cuanto confias en el output de la IA como parte de tu trabajo):
- Highly trust (4)
- Somewhat trust (3)
- Neither trust nor distrust (2) 
- Somewhat distrust (1)
- Highly distrust (0)

LabelEncoder con los valores indicados 

In [4980]:
# Primero trato los Nan
df['AIAcc'] = df['AIAcc'].fillna('Other')
print(df['AIAcc'].isna().sum())

0


In [4981]:
df['AIAcc'].unique()

array(['Other', 'Neither trust nor distrust', 'Somewhat trust',
       'Somewhat distrust', 'Highly trust', 'Highly distrust'],
      dtype=object)

In [4982]:
labels63 = {
    'Highly trust': 4, 
    'Somewhat trust': 3, 
    'Neither trust nor distrust': 2,
    'Somewhat distrust': 1, 
    'Highly distrust': 0, 
    'Other': -1
}
df['AIAcc'] = df['AIAcc'].map(labels63).fillna(-1)

64. AIComplex (Que tan bien manejan tareas complejas las herramientas de IA que usas):
- Very well at handling complex tasks (4)
- Good, but not great at handling complex tasks (3)
- Neither good or bad at handling complex tasks (2)
- Bad at handling complex tasks (1)
- Very poor at handling complex tasks (0)

LabelEncoder, con los valores indicados 

In [4983]:
# Primero trato los Nan
df['AIComplex'] = df['AIComplex'].fillna('Other')
print(df['AIComplex'].isna().sum())

0


In [4984]:
df['AIComplex'].unique()

array(['Other', 'Neither good or bad at handling complex tasks',
       'Good, but not great at handling complex tasks',
       'Bad at handling complex tasks',
       'Very well at handling complex tasks',
       'Very poor at handling complex tasks'], dtype=object)

In [4985]:
labels64 = {
    'Very well at handling complex tasks': 4, 
    'Good, but not great at handling complex tasks': 3, 
    'Neither good or bad at handling complex tasks': 2,
    'Bad at handling complex tasks': 1, 
    'Very poor at handling complex tasks': 0, 
    'Other': -1
}
df['AIComplex'] = df['AIComplex'].map(labels64).fillna(-1)

65. AIToolCurrently Using:
- Learning about a codebase 
- Project planning 
- Writing code 
- Documenting code 
- Debugging and getting help 
- Testing code 
- Committing and reviewing code 
- Deployment and monitoring 
- Predictive analytics 
- Search for answers 
- Generating content or synthetic data 
- Other 

Dividirlo por ';' y luego OneHotEncoder.

66. AIToolInterested in Using: La elimino, dejo solo la de "Currently in using" como representativa. Sino se me hacen muchas columnas y muy repetitivas

67. AIToolNot interested in Using: La elimino, dejo solo la de "Currently in using" como representativa. Sino se me hacen muchas columnas y muy repetitivas

68. AINextMuch more integrated (Cuales estarán mucho más integradas): 
- Learning about a codebase 
- Project planning 
- Writing code 
- Documenting code 
- Debugging and getting help 
- Testing code 
- Committing and reviewing code 
- Deployment and monitoring 
- Predictive analytics 
- Search for answers 
- Generating content or synthetic data 
- Other 

Dividirlo por ';' y luego OneHotEncoder. 

69. AINextNo change: La elimino, dejo solo la de "Much more integrated" como representativa. Sino se me hacen muchas columnas y muy repetitivas

70. AINextMore integrated: La elimino, dejo solo la de "Much more integrated" como representativa. Sino se me hacen muchas columnas y muy repetitivas

71. AINextLess integrated: La elimino, dejo solo la de "Much more integrated" como representativa. Sino se me hacen muchas columnas y muy repetitivas

72. AINextMuch less integrated: La elimino, dejo solo la de "Much more integrated" como representativa. Sino se me hacen muchas columnas y muy repetitivas

73. AIThreat: 
- Yes (2)
- No (0)
- I'm not sure (1)

LabelEncoder con los valores indicados 

In [4986]:
# Primero trato los Nan
df['AIThreat'] = df['AIThreat'].fillna("I'm not sure")
print(df['AIThreat'].isna().sum())

0


In [4987]:
df['AIThreat'].unique()

array(['No', 'Yes', "I'm not sure"], dtype=object)

In [4988]:
labels73 = {
    'Yes': 2,
    'No': 0, 
    "I'm not sure": 1, 
}
df['AIThreat'] = df['AIThreat'].map(labels73).fillna(-1)

74. AIEthics (Qué responsabilidades éticas de la IA son importantes para ti): No creo que sea muy relevante, la elimino. 

75. AIChallenges (Qué desafíos enfrenta tu equipo/compañía a la hora de usar asistentes de código con IA o GenAi):
- Don’t trust the output or answers 
- AI tools lack context of codebase, internal architecture, and/or company knowledge 
- They create more work (more code/PRs to review, etc.) 
- We don’t have the right policies in place to reduce security risks 
- Not everyone uses them 
- Lack of executive buy-in 
- Lack of proper training and education on new tools 
- None of the above 
- Other 

Dividirlo por ';' y luego OneHotEncoder.

76. TBranch: Irrelevante, la elimino

77. ICorPM: Irrelevante, la elimino

78. WorkExp: Los dejo como estan 

79. Knowledge_1: Irrelevante, lo elimino

80. Knowledge_2: Irrelevante, lo elimino

81. Knowledge_3: Irrelevante, lo elimino

82. Knowledge_4: Irrelevante, lo elimino

83. Knowledge_5: Irrelevante, lo elimino

84. Knowledge_6: Irrelevante, lo elimino

85. Knowledge_7: Irrelevante, lo elimino

86. Knowledge_8: Irrelevante, lo elimino

87. Knowledge_9: Irrelevante, lo elimino

88. Frequency_1 (Qué tan frecuente experimentas ‘Needing help from people outside of your immediate team?‘): LabelEncoder 

In [4989]:
# Primero trato los Nan
print(df['Frequency_1'].isna().sum())

246


In [4990]:
df['Frequency_1'].unique()

array([nan, '1-2 times a week', '3-5 times a week', 'Never',
       '6-10 times a week', '10+ times a week'], dtype=object)

In [4991]:
labels88 = {
    '10+ times a week': 4, 
    '6-10 times a week': 3, 
    '3-5 times a week': 2,
    '1-2 times a week': 1, 
    'Never': 0, 
    'Other': -1
}

median_freq1 = df['Frequency_1'].map(labels88).median()
df['Frequency_1'] = df['Frequency_1'].map(labels88).fillna(median_freq1)

89. Frequency_2 (Qué tan frecuente experimentas ‘Interacting with people outside of your immediate team?’):  LabelEncoder

Mismas categorias que la 88, uso ese labels

In [4992]:
df['Frequency_2'].unique()

array([nan, '3-5 times a week', '1-2 times a week', 'Never',
       '6-10 times a week', '10+ times a week'], dtype=object)

In [4993]:
# Primero trato los Nan

print(df['Frequency_2'].isna().sum())

247


In [4994]:
median_freq2 = df['Frequency_2'].map(labels88).median()
df['Frequency_2'] = df['Frequency_2'].map(labels88).fillna(median_freq2)

90. Frequency_3 (Qué tan frecuente experimentas ‘Encountering knowledge silos (where one individual or team has information that's not shared or distributed with other individuals or teams) at work?’): LabelEncoder

Mismas categorias que la 88, uso ese labels

In [4996]:
df['Frequency_3'].unique()

array([nan, '1-2 times a week', 'Never', '10+ times a week',
       '3-5 times a week', '6-10 times a week'], dtype=object)

In [4997]:
# Primero trato los Nan
# df['Frequency_3'] = df['Frequency_3'].fillna('Other')
print(df['Frequency_3'].isna().sum())

260


In [4998]:
median_freq3 = df['Frequency_3'].map(labels88).median()
df['Frequency_3'] = df['Frequency_3'].map(labels88).fillna(median_freq3)

91. TimeSearching (Promedio de tiempo que gastas buscando respuesta o solución a algún problema): 
- 'Less than 15 minutes a day': 0,
- '15-30 minutes a day': 1,
- '30-60 minutes a day': 2,
- '60-120 minutes a day': 3,
- 'Over 120 minutes a day': 4

LabelEncoder con los valores asignados

In [4999]:
df['TimeSearching'].unique()

array([nan, '15-30 minutes a day', 'Over 120 minutes a day',
       '30-60 minutes a day', 'Less than 15 minutes a day',
       '60-120 minutes a day'], dtype=object)

In [5000]:
# Primero trato los Nan
df['TimeSearching'] = df['TimeSearching'].fillna('IDK')
print(df['TimeSearching'].isna().sum())

0


In [5001]:
labels91 = {
'Less than 15 minutes a day': 0,
    '15-30 minutes a day': 1,
    '30-60 minutes a day': 2,
    '60-120 minutes a day': 3,
    'Over 120 minutes a day': 4,
    'IDK': -1
}
df['TimeSearching'] = df['TimeSearching'].map(labels91).fillna(-1)

92. TimeAnswering (Cuánto tiempo gastas en responder preguntas que te hacen?)

LabelEncoder con los valores asignados a la 91 (mismas respuestas)

In [5002]:
df['TimeAnswering'].unique()

array([nan, '30-60 minutes a day', 'Over 120 minutes a day',
       '15-30 minutes a day', '60-120 minutes a day',
       'Less than 15 minutes a day'], dtype=object)

In [5003]:
# Primero trato los Nan
df['TimeAnswering'] = df['TimeAnswering'].fillna('IDK')
print(df['TimeAnswering'].isna().sum())

0


In [5004]:
df['TimeAnswering'] = df['TimeAnswering'].map(labels91).fillna(-1)

93. Frustration: Qué desafío te causa mas frustración
- Complexity of tech stack for build 
- Complexity of tech stack for deployment 
- Amount of technical debt 
- Number of software tools in use 
- Patching/updating core components 
- Maintaining security of code being produced Maintaining security of systems/platforms used in work 
- Reliability of tools/systems used in work 
- Tracking my work 
- Showing my contributions 
- None of these

Dividirlo por ';' y luego OneHotEncoder. 

94. ProfessionalTech: Irrelevante, lo elimino.

95. ProfessionalCloud: Irrelevante, lo elimino

In [5006]:
df=  df.drop(['ProfessionalCloud'], axis=1)

96. ProfessionalQuestion: Irrelevante, lo elimino

In [5007]:
df=  df.drop(['ProfessionalQuestion'], axis=1)

97. Industry (Industria de tu compañía):
- Software Development (0)
- Computer Systems Design and Services (0)
- Internet, Telecomm or Information Services (0)
- Fintech (0)
- Energy (3)
- Government (4)
- Banking/Financial Services (1)
- Manufacturing (3)
- Transportation, or Supply Chain (3)
- Healthcare (2)
- Retail and Consumer Services (4)
- Higher Education (2)
- Media & Advertising Services (4)
- Insurance (1)
- Other(4)

Los agrupo y les hago OneHotEncoder. 

In [5008]:
df['Industry'].unique()

array([nan, 'Insurance', 'Healthcare',
       'Computer Systems Design and Services',
       'Media & Advertising Services', 'Software Development', 'Fintech',
       'Retail and Consumer Services',
       'Internet, Telecomm or Information Services', 'Other:', 'Energy',
       'Transportation, or Supply Chain', 'Banking/Financial Services',
       'Manufacturing', 'Higher Education', 'Government'], dtype=object)

In [5009]:
# Diccionario de mapeo
industry_map = {
    'Software Development': 'Tecnología y Servicios Digitales',
    'Computer Systems Design and Services': 'Tecnología y Servicios Digitales',
    'Internet, Telecomm or Information Services': 'Tecnología y Servicios Digitales',
    'Fintech': 'Tecnología y Servicios Digitales',
    'Energy': 'Industria y Energía',
    'Government': 'Otros Servicios',
    'Banking/Financial Services': 'Servicios Financieros',
    'Manufacturing': 'Industria y Energía',
    'Transportation, or Supply Chain': 'Industria y Energía',
    'Healthcare': 'Salud y Educación',
    'Retail and Consumer Services': 'Otros Servicios',
    'Higher Education': 'Salud y Educación',
    'Media & Advertising Services': 'Otros Servicios',
    'Insurance': 'Servicios Financieros',
    'Other:': 'Otros Servicios'
}

# Función para manejar NaN y aplicar el mapeo
def map_industry(value):
    if pd.isna(value):
        return 'Desconocido'
    return industry_map.get(value, 'Otros Servicios')

# Suponiendo que tienes un DataFrame llamado 'df' con una columna 'Industry'
# Aplicamos el mapeo a la columna 'Industry'
most_frequent_industry = df['Industry'].map(industry_map).mode()[0]
df['Industry_Category'] = df['Industry'].map(industry_map).fillna(most_frequent_industry)

# Creamos el OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Aplicamos el OneHotEncoder
encoded = encoder.fit_transform(df[['Industry_Category']])

# Creamos un DataFrame con las columnas codificadas
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['Industry_Category']), index=df.index)

# Unimos el DataFrame original con las nuevas columnas codificadas
df = pd.concat([df, encoded_df], axis=1)

# Eliminamos la columna temporal 'Industry_Category' si ya no la necesitas
df = df.drop(['Industry_Category', 'Industry'], axis=1)

Por la forma en que OneHotEncoder trata los Nan, se me siguen creando columnas con Industry_Category_nan. Simplemente la Elimino y esa fila quedara sin valores

Creo la funcion para aplicar OneHotEncoder a todas las columnas que quedan, a las que primero tengo que tratar haciendo split de las ;. 

In [5011]:
def process_and_encode(df, columns):
    for column in columns:
        # Separamos las categorías por el delimitador ";"
        df[column] = df[column].fillna('').str.split(';')

        # Creamos un objeto MultiLabelBinarizer
        mlb = MultiLabelBinarizer()

        # Aplicamos MultiLabelBinarizer a las categorías separadas
        encoded_values = mlb.fit_transform(df[column])

        # Creamos un DataFrame con los valores codificados
        encoded_df = pd.DataFrame(encoded_values, columns=[f"{column}_{c}" for c in mlb.classes_], index=df.index)

        # Añadimos los resultados al DataFrame copiado
        df = pd.concat([df, encoded_df], axis=1)

        # Eliminamos la columna original del DataFrame copiado
        df.drop(column, axis=1, inplace=True)

    return df

Los Nan que existan en estas columnas he decidido dejarlos, y cuando se aplique el OneHotEncoder se pondran todos 0 en las columnas que agrega. En caso de que no sea asi, eliminare las columnas que sean nan

In [5012]:
# Supongamos que estas son las columnas que contienen valores separados por ';'
columns_to_encode = ['TechDoc', 'BuyNewTool', 'TechEndorse',
    'LanguageHaveWorkedWith','LanguageWantToWorkWith','DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith', 'EmbeddedHaveWorkedWith', 
    'MiscTechHaveWorkedWith', 'ToolsTechHaveWorkedWith', 'NEWCollabToolsHaveWorkedWith', 'OfficeStackAsyncHaveWorkedWith', 'OfficeStackSyncHaveWorkedWith',
    'AISearchDevHaveWorkedWith', 'AIBen', 'AIToolCurrently Using', 'AINextMuch more integrated', 'AIChallenges', 'Frustration'
    ]  # Reemplaza con los nombres reales de las columnas


# Aplicamos la función al DataFrame
df = process_and_encode(df, columns_to_encode)

In [5013]:
columns_to_drop = [ 'LanguageAdmired',
    'DatabaseWantToWorkWith', 'DatabaseAdmired',
    'PlatformWantToWorkWith', 'PlatformAdmired',
    'WebframeWantToWorkWith', 'WebframeAdmired',
    'EmbeddedWantToWorkWith', 'EmbeddedAdmired', 
    'MiscTechWantToWorkWith', 'MiscTechAdmired',
    'ToolsTechWantToWorkWith', 'ToolsTechAdmired', 
    'NEWCollabToolsWantToWorkWith', 'NEWCollabToolsAdmired', 
    'OpSysPersonal use',
    'OfficeStackAsyncWantToWorkWith', 'OfficeStackAsyncAdmired', 
    'OfficeStackSyncWantToWorkWith', 'OfficeStackSyncAdmired',
    'AISearchDevWantToWorkWith', 'AISearchDevAdmired',
    'NEWSOSites', 'SOVisitFreq', 'SOAccount', 'SOPartFreq', 'SOHow', 'SOComm', 
    'AIToolInterested in Using', 'AIToolNot interested in Using',
    'AINextNo change', 'AINextMore integrated', 'AINextLess integrated', 'AINextMuch less integrated',
    'AIEthics', 'TBranch', 'ICorPM', 'WorkExp', 
    'Knowledge_1', 'Knowledge_2', 'Knowledge_3', 'Knowledge_4', 'Knowledge_5', 'Knowledge_6', 'Knowledge_7', 'Knowledge_8', 'Knowledge_9', 
    'ProfessionalTech', 'SurveyLength', 'SurveyEase'
]

98. JobSatPoints_1: Elimino, irrelevante
99. JobSatPoints_4: Elimino, irrelevante

100. JobSatPoints_5: Elimino, irrelevante

101. JobSatPoints_6: Elimino, irrelevante

102. JobSatPoints_7: Elimino, irrelevante
102. JobSatPoints_8: Elimino, irrelevante
103. JobSatPoints_9: Elimino, irrelevante

104. JobSatPoints_10: Elimino, irrelevante
105. JobSatPoints_11: Elimino, irrelevante
106. JobSat: Elimino, irrelevante

In [5014]:
df = df.drop(['JobSatPoints_1', 'JobSatPoints_4', 'JobSatPoints_5', 'JobSatPoints_6', 'JobSatPoints_7', 
               'JobSatPoints_8', 'JobSatPoints_9', 'JobSatPoints_10', 'JobSatPoints_11' ,'JobSat'], 
             axis=1)

Elimino las columas que estaban en columns_to_drop

In [5015]:
df = df.drop(columns_to_drop, axis=1)

In [5017]:
for col in df.columns:
    print(col) 

MainBranch
EdLevel
YearsCode
YearsCodePro
PurchaseInfluence
CompTotal
AISelect
AISent
AIAcc
AIComplex
AIThreat
Frequency_1
Frequency_2
Frequency_3
TimeSearching
TimeAnswering
Age_Grouped_Adulto
Age_Grouped_Desconocido
Age_Grouped_Joven
Age_Grouped_Senior
is_full_time
is_part_time
is_independent
num_jobs
is_other_employment
Remote_grouped_In-person
Remote_grouped_Remote/Hybrid
CodingActivities_encoded
LearnCode_encoded
LearnCodeOnline_encoded
DevType_encoded
OrgSize_Grouped_Desconocido
OrgSize_Grouped_Grande
OrgSize_Grouped_Mediana
OrgSize_Grouped_Muy Pequeña
OrgSize_Grouped_Pequeña
Android-based
Linux-based
MacOS
Otros
Windows-based
Industry_Category_Industria y Energía
Industry_Category_Otros Servicios
Industry_Category_Salud y Educación
Industry_Category_Servicios Financieros
Industry_Category_Tecnología y Servicios Digitales
TechDoc_
TechDoc_AI-powered search/dev tool (free)
TechDoc_AI-powered search/dev tool (paid)
TechDoc_API document(s) and/or SDK document(s)
TechDoc_First-party 

In [5018]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784 entries, 0 to 783
Columns: 527 entries, MainBranch to Frustration_Tracking my work
dtypes: float64(24), int32(493), int64(10)
memory usage: 1.7 MB


In [5019]:
df.head()

,MainBranch,EdLevel,YearsCode,YearsCodePro,PurchaseInfluence,CompTotal,AISelect,AISent,AIAcc,AIComplex,AIThreat,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Age_Grouped_Adulto,Age_Grouped_Desconocido,Age_Grouped_Joven,Age_Grouped_Senior,is_full_time,is_part_time,is_independent,num_jobs,is_other_employment,Remote_grouped_In-person,Remote_grouped_Remote/Hybrid,CodingActivities_encoded,LearnCode_encoded,LearnCodeOnline_encoded,DevType_encoded,OrgSize_Grouped_Desconocido,OrgSize_Grouped_Grande,OrgSize_Grouped_Mediana,OrgSize_Grouped_Muy Pequeña,OrgSize_Grouped_Pequeña,Android-based,Linux-based,MacOS,Otros,Windows-based,Industry_Category_Industria y Energía,Industry_Category_Otros Servicios,Industry_Category_Salud y Educación,Industry_Category_Servicios Financieros,Industry_Category_Tecnología y Servicios Digitales,TechDoc_,TechDoc_AI-powered search/dev tool (free),TechDoc_AI-powered search/dev tool (paid),TechDoc_API document(s) and/or SDK document(s),TechDoc_First-party knowledge base,TechDoc_Other (please specify):,TechDoc_Traditional public search engine,TechDoc_User guides or README files found in the source repository,BuyNewTool_Ask a generative AI tool,BuyNewTool_Ask developers I know/work with,BuyNewTool_Other,BuyNewTool_Other (please specify):,BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd,BuyNewTool_Research companies that have advertised on sites I visit,BuyNewTool_Research companies that have emailed me,BuyNewTool_Start a free trial,BuyNewTool_Visit developer communities like Stack Overflow,TechEndorse_AI tool integration,TechEndorse_APIs,TechEndorse_Connected to an open source project,TechEndorse_Customization,TechEndorse_Frequent updates to features,TechEndorse_Integrated developer community,TechEndorse_Other,TechEndorse_Other:,TechEndorse_Quality support system,TechEndorse_Reputation for quality and excellence,LanguageHaveWorkedWith_,LanguageHaveWorkedWith_Ada,LanguageHaveWorkedWith_Apex,LanguageHaveWorkedWith_Assembly,LanguageHaveWorkedWith_Bash/Shell (all shells),LanguageHaveWorkedWith_C,LanguageHaveWorkedWith_C#,LanguageHaveWorkedWith_C++,LanguageHaveWorkedWith_Clojure,LanguageHaveWorkedWith_Cobol,LanguageHaveWorkedWith_Crystal,LanguageHaveWorkedWith_Dart,LanguageHaveWorkedWith_Delphi,LanguageHaveWorkedWith_Elixir,LanguageHaveWorkedWith_Erlang,LanguageHaveWorkedWith_F#,LanguageHaveWorkedWith_Fortran,LanguageHaveWorkedWith_GDScript,LanguageHaveWorkedWith_Go,LanguageHaveWorkedWith_Groovy,LanguageHaveWorkedWith_HTML/CSS,LanguageHaveWorkedWith_Haskell,LanguageHaveWorkedWith_Java,LanguageHaveWorkedWith_JavaScript,LanguageHaveWorkedWith_Julia,LanguageHaveWorkedWith_Kotlin,LanguageHaveWorkedWith_Lisp,LanguageHaveWorkedWith_Lua,LanguageHaveWorkedWith_MATLAB,LanguageHaveWorkedWith_MicroPython,LanguageHaveWorkedWith_Nim,LanguageHaveWorkedWith_OCaml,LanguageHaveWorkedWith_Objective-C,LanguageHaveWorkedWith_PHP,LanguageHaveWorkedWith_Perl,LanguageHaveWorkedWith_PowerShell,LanguageHaveWorkedWith_Prolog,LanguageHaveWorkedWith_Python,LanguageHaveWorkedWith_R,LanguageHaveWorkedWith_Ruby,LanguageHaveWorkedWith_Rust,LanguageHaveWorkedWith_SQL,LanguageHaveWorkedWith_Scala,LanguageHaveWorkedWith_Solidity,LanguageHaveWorkedWith_Swift,LanguageHaveWorkedWith_TypeScript,LanguageHaveWorkedWith_VBA,LanguageHaveWorkedWith_Visual Basic (.Net),LanguageHaveWorkedWith_Zig,LanguageWantToWorkWith_,LanguageWantToWorkWith_Ada,LanguageWantToWorkWith_Apex,LanguageWantToWorkWith_Assembly,LanguageWantToWorkWith_Bash/Shell (all shells),LanguageWantToWorkWith_C,LanguageWantToWorkWith_C#,LanguageWantToWorkWith_C++,LanguageWantToWorkWith_Clojure,LanguageWantToWorkWith_Crystal,LanguageWantToWorkWith_Dart,LanguageWantToWorkWith_Delphi,LanguageWantToWorkWith_Elixir,LanguageWantToWorkWith_Erlang,LanguageWantToWorkWith_F#,LanguageWantToWorkWith_Fortran,LanguageWantToWorkWith_GDScript,LanguageWantToWorkWith_Go,LanguageWantToWorkWith_Groovy,LanguageWantToWorkWith_HTML/CSS,LanguageWantToWorkWith_Haskell,LanguageWantToWorkW

## Me traigo el df del 2023, ya limpio y con numeros, para hacerle concat a este

In [5020]:
# Cargar el DataFrame desde el archivo
with open('../Pickles/data_23.pickle', 'rb') as archivo:
    df1 = pickle.load(archivo)

In [5021]:
df1

,MainBranch,EdLevel,YearsCode,YearsCodePro,PurchaseInfluence,CompTotal,AISelect,AISent,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Age_Grouped_Adulto,Age_Grouped_Joven,Age_Grouped_Senior,is_full_time,is_part_time,is_independent,num_jobs,is_other_employment,Remote_grouped_In-person,Remote_grouped_Remote/Hybrid,CodingActivities_encoded,LearnCode_encoded,LearnCodeOnline_encoded,DevType_encoded,OrgSize_Grouped_Desconocido,OrgSize_Grouped_Grande,OrgSize_Grouped_Mediana,OrgSize_Grouped_Muy Pequeña,OrgSize_Grouped_Pequeña,Android-based,Linux-based,MacOS,Otros,Windows-based,Industry_Category_Industria y Energía,Industry_Category_Otros Servicios,Industry_Category_Salud y Educación,Industry_Category_Servicios Financieros,Industry_Category_Tecnología y Servicios Digitales,BuyNewTool_Ask a generative AI tool,BuyNewTool_Ask developers I know/work with,BuyNewTool_Other,BuyNewTool_Other (please specify):,BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd,BuyNewTool_Research companies that have advertised on sites I visit,BuyNewTool_Research companies that have emailed me,BuyNewTool_Start a free trial,BuyNewTool_Visit developer communities like Stack Overflow,LanguageHaveWorkedWith_,LanguageHaveWorkedWith_Ada,LanguageHaveWorkedWith_Apex,LanguageHaveWorkedWith_Assembly,LanguageHaveWorkedWith_Bash/Shell (all shells),LanguageHaveWorkedWith_C,LanguageHaveWorkedWith_C#,LanguageHaveWorkedWith_C++,LanguageHaveWorkedWith_Clojure,LanguageHaveWorkedWith_Cobol,LanguageHaveWorkedWith_Crystal,LanguageHaveWorkedWith_Dart,LanguageHaveWorkedWith_Delphi,LanguageHaveWorkedWith_Elixir,LanguageHaveWorkedWith_Erlang,LanguageHaveWorkedWith_F#,LanguageHaveWorkedWith_Flow,LanguageHaveWorkedWith_Fortran,LanguageHaveWorkedWith_GDScript,LanguageHaveWorkedWith_Go,LanguageHaveWorkedWith_Groovy,LanguageHaveWorkedWith_HTML/CSS,LanguageHaveWorkedWith_Haskell,LanguageHaveWorkedWith_Java,LanguageHaveWorkedWith_JavaScript,LanguageHaveWorkedWith_Julia,LanguageHaveWorkedWith_Kotlin,LanguageHaveWorkedWith_Lisp,LanguageHaveWorkedWith_Lua,LanguageHaveWorkedWith_MATLAB,LanguageHaveWorkedWith_Nim,LanguageHaveWorkedWith_OCaml,LanguageHaveWorkedWith_Objective-C,LanguageHaveWorkedWith_PHP,LanguageHaveWorkedWith_Perl,LanguageHaveWorkedWith_PowerShell,LanguageHaveWorkedWith_Prolog,LanguageHaveWorkedWith_Python,LanguageHaveWorkedWith_R,LanguageHaveWorkedWith_Raku,LanguageHaveWorkedWith_Ruby,LanguageHaveWorkedWith_Rust,LanguageHaveWorkedWith_SAS,LanguageHaveWorkedWith_SQL,LanguageHaveWorkedWith_Scala,LanguageHaveWorkedWith_Solidity,LanguageHaveWorkedWith_Swift,LanguageHaveWorkedWith_TypeScript,LanguageHaveWorkedWith_VBA,LanguageHaveWorkedWith_Visual Basic (.Net),LanguageHaveWorkedWith_Zig,LanguageWantToWorkWith_,LanguageWantToWorkWith_APL,LanguageWantToWorkWith_Ada,LanguageWantToWorkWith_Apex,LanguageWantToWorkWith_Assembly,LanguageWantToWorkWith_Bash/Shell (all shells),LanguageWantToWorkWith_C,LanguageWantToWorkWith_C#,LanguageWantToWorkWith_C++,LanguageWantToWorkWith_Clojure,LanguageWantToWorkWith_Cobol,LanguageWantToWorkWith_Crystal,LanguageWantToWorkWith_Dart,LanguageWantToWorkWith_Delphi,LanguageWantToWorkWith_Elixir,LanguageWantToWorkWith_Erlang,LanguageWantToWorkWith_F#,LanguageWantToWorkWith_Flow,LanguageWantToWorkWith_Fortran,LanguageWantToWorkWith_GDScript,LanguageWantToWorkWith_Go,LanguageWantToWorkWith_Groovy,LanguageWantToWorkWith_HTML/CSS,LanguageWantToWorkWith_Haskell,LanguageWantToWorkWith_Java,LanguageWantToWorkWith_JavaScript,LanguageWantToWorkWith_Julia,LanguageWantToWorkWith_Kotlin,LanguageWantToWorkWith_Lisp,LanguageWantToWorkWith_Lua,LanguageWantToWorkWith_MATLAB,LanguageWantToWorkWith_Nim,LanguageWantToWorkWith_OCaml,LanguageWantToWorkWith_Objective-C,LanguageWantToWorkWith_PHP,LanguageWantToWorkWith_Perl,LanguageWantToWorkWith_PowerShell,LanguageWantToWorkWith_Prolog,LanguageWantToWorkWith_Python,LanguageWantToWorkWith_R,LanguageWantToWorkWith_Raku,LanguageWantToWorkWith_Ruby,LanguageWantToWorkWith_Rust,LanguageWantToWorkWith_SA

In [5022]:
# Combinar los DataFrames usando pd.concat()
df_merged = pd.concat([df1, df], ignore_index=True, join='inner')

In [5023]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1934 entries, 0 to 1933
Columns: 397 entries, MainBranch to AIToolCurrently Using_Writing code
dtypes: float64(23), int32(367), int64(7)
memory usage: 3.2 MB


In [5024]:
df_merged 

,MainBranch,EdLevel,YearsCode,YearsCodePro,PurchaseInfluence,CompTotal,AISelect,AISent,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Age_Grouped_Adulto,Age_Grouped_Joven,Age_Grouped_Senior,is_full_time,is_part_time,is_independent,num_jobs,is_other_employment,Remote_grouped_In-person,Remote_grouped_Remote/Hybrid,CodingActivities_encoded,LearnCode_encoded,LearnCodeOnline_encoded,DevType_encoded,OrgSize_Grouped_Desconocido,OrgSize_Grouped_Grande,OrgSize_Grouped_Mediana,OrgSize_Grouped_Muy Pequeña,OrgSize_Grouped_Pequeña,Android-based,Linux-based,MacOS,Otros,Windows-based,Industry_Category_Industria y Energía,Industry_Category_Otros Servicios,Industry_Category_Salud y Educación,Industry_Category_Servicios Financieros,Industry_Category_Tecnología y Servicios Digitales,BuyNewTool_Ask a generative AI tool,BuyNewTool_Ask developers I know/work with,BuyNewTool_Other,BuyNewTool_Other (please specify):,BuyNewTool_Read ratings or reviews on third party sites like G2 Crowd,BuyNewTool_Research companies that have advertised on sites I visit,BuyNewTool_Research companies that have emailed me,BuyNewTool_Start a free trial,BuyNewTool_Visit developer communities like Stack Overflow,LanguageHaveWorkedWith_,LanguageHaveWorkedWith_Ada,LanguageHaveWorkedWith_Apex,LanguageHaveWorkedWith_Assembly,LanguageHaveWorkedWith_Bash/Shell (all shells),LanguageHaveWorkedWith_C,LanguageHaveWorkedWith_C#,LanguageHaveWorkedWith_C++,LanguageHaveWorkedWith_Clojure,LanguageHaveWorkedWith_Cobol,LanguageHaveWorkedWith_Crystal,LanguageHaveWorkedWith_Dart,LanguageHaveWorkedWith_Delphi,LanguageHaveWorkedWith_Elixir,LanguageHaveWorkedWith_Erlang,LanguageHaveWorkedWith_F#,LanguageHaveWorkedWith_Fortran,LanguageHaveWorkedWith_GDScript,LanguageHaveWorkedWith_Go,LanguageHaveWorkedWith_Groovy,LanguageHaveWorkedWith_HTML/CSS,LanguageHaveWorkedWith_Haskell,LanguageHaveWorkedWith_Java,LanguageHaveWorkedWith_JavaScript,LanguageHaveWorkedWith_Julia,LanguageHaveWorkedWith_Kotlin,LanguageHaveWorkedWith_Lisp,LanguageHaveWorkedWith_Lua,LanguageHaveWorkedWith_MATLAB,LanguageHaveWorkedWith_Nim,LanguageHaveWorkedWith_OCaml,LanguageHaveWorkedWith_Objective-C,LanguageHaveWorkedWith_PHP,LanguageHaveWorkedWith_Perl,LanguageHaveWorkedWith_PowerShell,LanguageHaveWorkedWith_Prolog,LanguageHaveWorkedWith_Python,LanguageHaveWorkedWith_R,LanguageHaveWorkedWith_Ruby,LanguageHaveWorkedWith_Rust,LanguageHaveWorkedWith_SQL,LanguageHaveWorkedWith_Scala,LanguageHaveWorkedWith_Solidity,LanguageHaveWorkedWith_Swift,LanguageHaveWorkedWith_TypeScript,LanguageHaveWorkedWith_VBA,LanguageHaveWorkedWith_Visual Basic (.Net),LanguageHaveWorkedWith_Zig,LanguageWantToWorkWith_,LanguageWantToWorkWith_Ada,LanguageWantToWorkWith_Apex,LanguageWantToWorkWith_Assembly,LanguageWantToWorkWith_Bash/Shell (all shells),LanguageWantToWorkWith_C,LanguageWantToWorkWith_C#,LanguageWantToWorkWith_C++,LanguageWantToWorkWith_Clojure,LanguageWantToWorkWith_Crystal,LanguageWantToWorkWith_Dart,LanguageWantToWorkWith_Delphi,LanguageWantToWorkWith_Elixir,LanguageWantToWorkWith_Erlang,LanguageWantToWorkWith_F#,LanguageWantToWorkWith_Fortran,LanguageWantToWorkWith_GDScript,LanguageWantToWorkWith_Go,LanguageWantToWorkWith_Groovy,LanguageWantToWorkWith_HTML/CSS,LanguageWantToWorkWith_Haskell,LanguageWantToWorkWith_Java,LanguageWantToWorkWith_JavaScript,LanguageWantToWorkWith_Julia,LanguageWantToWorkWith_Kotlin,LanguageWantToWorkWith_Lisp,LanguageWantToWorkWith_Lua,LanguageWantToWorkWith_MATLAB,LanguageWantToWorkWith_Nim,LanguageWantToWorkWith_OCaml,LanguageWantToWorkWith_Objective-C,LanguageWantToWorkWith_PHP,LanguageWantToWorkWith_Perl,LanguageWantToWorkWith_PowerShell,LanguageWantToWorkWith_Prolog,LanguageWantToWorkWith_Python,LanguageWantToWorkWith_R,LanguageWantToWorkWith_Ruby,LanguageWantToWorkWith_Rust,LanguageWantToWorkWith_SQL,LanguageWantToWorkWith_Scala,LanguageWantToWorkWith_Solidity,LanguageWantToWorkWith_Swift,LanguageWantToWorkWith_TypeScript,LanguageWantToWorkWith_VBA,LanguageWantToWorkWith_Visual Basic (.Net)

Ya tengo toda la data lista para terminar de hacer feature enegineering y empezar con el modelo. 
Lo exporto, sigo en otro notebook

In [5025]:
# Construir la ruta relativa a la carpeta Pickles
ruta_pickles = os.path.join("..", "Pickles")

# Asegurarse de que la carpeta Pickles exista
os.makedirs(ruta_pickles, exist_ok=True)

# Guardar los DataFrames en formato pickle en la carpeta Pickles
with open(os.path.join(ruta_pickles, 'df_final.pickle'), 'wb') as archivo:
    pickle.dump(df_merged, archivo)